# Projekt

## 1. Zielsetzung und Datenverständnis

In diesem Projekt wenden wir die Konzepte aus den vorherigen Notebooks auf einen echten Datensatz an. Ziel ist es, mithilfe der bisher erlernten Methoden ein einfaches maschinelles Lernmodell zu entwickeln, das auf Basis physikalischer und materialtechnischer Eigenschaften vorhersagen kann, ob eine Steckverbindung im Laufe der Zeit einen kritischen Widerstand überschreiten wird.

### Der Kontext

Steckverbindungen sind essenzielle Bauteile in vielen technischen Systemen – von der Automobilindustrie bis zur Elektronikfertigung. Mit der Zeit und unter Belastung (z. B. durch Vibration oder Temperaturwechsel) kann der elektrische Kontaktwiderstand steigen. Überschreitet er einen bestimmten Schwellenwert (meist 300 mΩ), gilt die Verbindung typischerweise als defekt.

Der vorliegende Datensatz enthält Informationen zu mehreren Steckverbindungen, darunter:

* **Materialeigenschaften** wie die Art der Beschichtung (Silber, Zinn) oder das Vorhandensein einer Zwischenschicht (Nickel),
* **Prozessparameter** wie Normalkraft, Frequenz und Bewegungshub,
* sowie die **Zyklenzahl**, bei der ein bestimmter Widerstandsschwellenwert (z. B. 1 mΩ, 20 mΩ, …, 300 mΩ) erstmals überschritten wurde – falls überhaupt.

### Das Ziel

Ihr werdet ein Modell entwickeln, das vorhersagt:

> **Wird eine Steckverbindung im Laufe der Belastungszyklen einen Widerstand von 300 mΩ überschreiten?**

Diese Frage ist eine **binäre Klassifikation**, d. h. wir möchten eine Ja-/Nein-Aussage (1/0) treffen:

* **Ja/1:** Der Widerstand wird irgendwann ≥ 300 mΩ.
* **Nein/0:** Die Schwelle wird nie überschritten.

### Der Datensatz

Die Datei `Schwellenwerte-Table 1.csv` enthält pro Zeile eine Steckverbindung mit:

* einem **Dateinamen** (`Datei`)
* Angaben zu **Material und Prozessparametern**
* Zyklenzahlen, bei denen bestimmte **Ohm-Schwellen überschritten wurden** (z. B. `Zyklus_300_mOhm`)

  * Ein Wert von **0** bedeutet: Schwelle wurde **nicht** überschritten.
  * Ein **NaN** bedeutet: Keine Messung oder keine Angabe vorhanden.


## Aufgabe 1: Überblick verschaffen

Lade die Datei `Schwellenwerte-Table 1.csv` mit `pandas` und verschaffe dir einen Überblick:

* Gib die Form des DataFrames mit `.shape` aus
* Zeige die ersten 5 Zeilen mit `.head()`
* Liste die Spaltennamen auf
* Gib eine Übersicht der statistischen Kenngrößen aus (max, min, std, mean, ...)

In [1]:
### Importiere pandas
import pandas as pd

# Lade die CSV-Datei in einen DataFrame
df = pd.read_csv('../Data/Schwellenwerte-Table 1.csv')

# Zeige die Form des DataFrames, die ersten 5 Zeilen, die Spaltennamen und eine Übersicht der statistischen Kenngrößen an
print(df.shape)

# Zeige die ersten 5 Zeilen des DataFrames
print(df.head())

# Liste die Spaltennamen auf
print(df.columns)

# Gib eine Übersicht der statistischen Kenngrößen aus
print(df.describe())


(99, 14)
                                Datei Beschichtung Zwischenschicht  \
0    AgCu-200µm-1N-75000zyklen-V1.txt        Keine           Keine   
1   AgCu-200µm-2.5N-6100zyklen-V1.txt        Keine           Keine   
2  AgCu-200µm-2.5N-29000zyklen-V1.txt        Keine           Keine   
3     AgCu-200µm-5N-3000zyklen-V1.txt        Keine           Keine   
4    AgCu-200µm-5N-13000zyklen-V1.txt        Keine           Keine   

   Normalkraft  Frequenz  Bewegungshub  Zyklus_1_mOhm  Zyklus_2_mOhm  \
0          1.0         1           200            0.0           16.0   
1          2.5         1           200            0.0            0.0   
2          2.5         1           200            0.0         6810.0   
3          5.0         1           200            0.0            NaN   
4          5.0         1           200            0.0            NaN   

   Zyklus_5_mOhm  Zyklus_10_mOhm  Zyklus_20_mOhm  Zyklus_50_mOhm  \
0        15360.0         42482.0         50456.0         50729.0   
1

## Aufgabe 2: Fehlende Werte handlen und Datentypen überprüfen

Untersuche die fehlenden Werte:

* Gib die Anzahl der fehlenden Werte pro Spalte aus.
* Gib die Datentypen aller Spalten mit `.dtypes` aus.

In [2]:
# Untersuche die fehlenden Werte
print(df.isna().sum())

# Zeige die Datentypen aller Spalten
print(df.dtypes)

Datei                 0
Beschichtung_Ag_Sn    0
Zwischenschicht_Ni    0
Normalkraft           0
Frequenz              0
Bewegungshub          0
Zyklus_1_mOhm         0
Zyklus_2_mOhm         0
Zyklus_5_mOhm         1
Zyklus_10_mOhm        0
Zyklus_20_mOhm        0
Zyklus_50_mOhm        0
Zyklus_100_mOhm       0
Zyklus_300_mOhm       0
dtype: int64
Datei                  object
Beschichtung_Ag_Sn      int64
Zwischenschicht_Ni      int64
Normalkraft           float64
Frequenz                int64
Bewegungshub            int64
Zyklus_1_mOhm          object
Zyklus_2_mOhm          object
Zyklus_5_mOhm          object
Zyklus_10_mOhm         object
Zyklus_20_mOhm         object
Zyklus_50_mOhm         object
Zyklus_100_mOhm        object
Zyklus_300_mOhm        object
dtype: object


In [3]:
from scripts.checker import check_missing_and_types

check_missing_and_types(df)

✅ Die fehlenden Werte sind:


Zyklus_5_mOhm    1
dtype: int64


✅ Die Datentypen sind:


Datei                  object
Beschichtung_Ag_Sn      int64
Zwischenschicht_Ni      int64
Normalkraft           float64
Frequenz                int64
Bewegungshub            int64
Zyklus_1_mOhm          object
Zyklus_2_mOhm          object
Zyklus_5_mOhm          object
Zyklus_10_mOhm         object
Zyklus_20_mOhm         object
Zyklus_50_mOhm         object
Zyklus_100_mOhm        object
Zyklus_300_mOhm        object
dtype: object

## Aufgabe 3: Zielvariable erstellen

Erstelle eine Spalte namens `target`, die wie folgt definiert ist:

* **1**, wenn `Zyklus_300_mOhm` einen Wert größer als 0 enthält
* **0**, wenn der Wert 0 oder NaN ist

In [2]:
df["target"] = (df["Zyklus_300_mOhm"] > 0).astype(int)

In [3]:
from scripts.checker import check_target_column
check_target_column(df)

✅ Zielvariable korrekt erstellt: 
     target
0        1
1        0
2        1
3        0
4        0
..     ...
94       1
95       0
96       0
97       0
98       0

[99 rows x 1 columns]


## Aufgabe 4: One-Hot-Encoding und Standardisierung

Bereite den Datensatz für das Modell vor:

1. Wandle **kategorische Variablen** in numerische mit One-Hot-Encoding um
2. Standardisiere **alle numerischen Features**, z. B. mit `StandardScaler` von scikit-learn
3. Speichere die verarbeiteten Features in einer neuen Variable `X_processed`

In [6]:
from sklearn.preprocessing import StandardScaler

# Features
features = ["Beschichtung", "Zwischenschicht", "Normalkraft", "Frequenz", "Bewegungshub", "Zyklus_1_mOhm", "Zyklus_2_mOhm", "Zyklus_5_mOhm", "Zyklus_10_mOhm", "Zyklus_20_mOhm", "Zyklus_50_mOhm", "Zyklus_100_mOhm", "Zyklus_300_mOhm"]

# Nur relevante Features extrahieren
X = df[features].copy()

# One-Hot-Encoding für kategorische Variablen
X_encoded = pd.get_dummies(X, columns=["Beschichtung", "Zwischenschicht"], drop_first=True)

# Standardisierung numerischer Spalten
numerical_cols = ["Normalkraft", "Frequenz", "Bewegungshub", "Zyklus_1_mOhm", "Zyklus_2_mOhm", "Zyklus_5_mOhm", "Zyklus_10_mOhm", "Zyklus_20_mOhm", "Zyklus_50_mOhm", "Zyklus_100_mOhm", "Zyklus_300_mOhm"]
scaler = StandardScaler()
X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

# Fertige Matrix
X_prepared = X_encoded


In [7]:
print(X_prepared.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Normalkraft           99 non-null     float64
 1   Frequenz              99 non-null     float64
 2   Bewegungshub          99 non-null     float64
 3   Zyklus_1_mOhm         97 non-null     float64
 4   Zyklus_2_mOhm         92 non-null     float64
 5   Zyklus_5_mOhm         89 non-null     float64
 6   Zyklus_10_mOhm        74 non-null     float64
 7   Zyklus_20_mOhm        63 non-null     float64
 8   Zyklus_50_mOhm        56 non-null     float64
 9   Zyklus_100_mOhm       45 non-null     float64
 10  Zyklus_300_mOhm       30 non-null     float64
 11  Beschichtung_Ag_Sn_0  99 non-null     bool   
 12  Beschichtung_Ag_Sn_1  99 non-null     bool   
 13  Zwischenschicht_Ni_0  99 non-null     bool   
 14  Zwischenschicht_Ni_1  99 non-null     bool   
dtypes: bool(4), float64(11)
m

In [5]:
from scripts.checker import check_preprocessing
check_preprocessing(X_prepared)

✅ One-Hot-Encoding & Standardisierung korrekt!


## Aufgabe 5: Daten in Trainings- und Testmenge aufteilen

Teile die Daten aus `X_prepared` und `df["target"]` in Trainings- und Testdaten auf.

Anforderungen:
* 80 % Training, 20 % Test
* Speichere die Arrays als:
  - `X_train`, `X_test`
  - `y_train`, `y_test`
* Setze `random_state=42` für Reproduzierbarkeit

In [ ]:
### Hier ist Platz für eure Lösung

In [ ]:
### Überprüft eure Lösung
from scripts.checker import check_split
# check_split(<>Hier eure X_train, X_test, y_train, y_test einfügen<>)

## Aufgabe 6: Logistisches Regressionsmodell trainieren

Trainiere ein logistisches Regressionsmodell auf den Trainingsdaten.

Anforderungen:
* Verwende `LogisticRegression` aus `sklearn.linear_model`
* Speichere das Modell in einer Variable `model`
* Trainiere mit `.fit(X_train, y_train)`

In [ ]:
### Hier ist Platz für eure Lösung

In [ ]:
### Überprüft eure Lösung
from scripts.checker import check_model_training
# check_model_training(<>Hier euer model, X_train, y_train einfügen<>)

## Aufgabe 7: Scatterplot der Vorhersagen

Visualisiere die Modellvorhersagen mithilfe eines Scatterplots:

Schritte:
1. Wähle zwei numerische Features, z. B. `Kontaktkraft_N` und `Steckzyklen`
2. Plotte die Testdaten in diesen zwei Dimensionen
3. Farbe: Nutze `model.predict(X_test)` als Farbe (0 oder 1)


In [ ]:
### Hier ist Platz für eure Lösung

## Aufgabe 8: Modell evaluieren

Bewerte die Qualität deiner Vorhersagen auf dem Testdatensatz.

Berechne folgende Metriken mit scikit-learn:

- Accuracy
- Precision
- Recall
- F1-Score
- Confusion Matrix
- ROC-AUC

> Hinweis: Verwende `model.predict(X_test)` für die Klassenvorhersage und `model.predict_proba(X_test)[:, 1]` für Wahrscheinlichkeiten.

Speichere die Vorhersagen in `y_pred` und die Wahrscheinlichkeiten in `y_prob`.

In [ ]:
### Hier ist Platz für eure Lösung

In [ ]:
### Überprüft eure Lösung
from scripts.checker import check_metrics
# check_metrics(<>Hier eure y_test, y_pred, y_prob einfügen<>)

## Aufgabe 9: Interpretation der Feature-Gewichte

Nutze die Koeffizienten des Modells, um zu verstehen, welche Features das Modell beeinflussen.

Schritte:
1. Erstelle ein DataFrame, das die Features (`X_prepared.columns`) den zugehörigen Koeffizienten (`model.coef_`) gegenüberstellt.
2. Sortiere es nach der Stärke des Einflusses (positiv oder negativ).
3. Stelle die Top-10 wichtigsten Features in einem Balkendiagramm dar.


In [ ]:
### Hier ist Platz für eure Lösung

In [ ]:
### Überprüft eure Lösung
from scripts.checker import check_coefficients
# check_coefficients(<>Hier euer model, X_prepared einfügen<>)